### 이미지 데이터셋 만들기(라벨 정보 없음)

In [2]:
import os
from glob import glob

import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt

In [4]:
# 업로드 concrete_image.zip 파일 확인
# The function returns a list of all files in the current working directory that match pattern
glob('concrete_image.zip')

[]

In [6]:
# 이미지 폴더 생성 및 concrete_image.zip 파일 압축 풀기 > 3,000개 이미지로 실습을 위해 데이터 축소

if not os.path.exists('IMAGE'):
    os.mkdir('IMAGE')
    !unzip concrete_image.zip -d IMAGE

unzip:  cannot find or open concrete_image.zip, concrete_image.zip.zip or concrete_image.zip.ZIP.


In [ ]:
# concrete_image Negative 폴더 안의 이미지 갯수
!ls -l ./IMAGE/Negative/ | grep jpg | wc -l

In [ ]:
# concrete_image Positive 폴더 안의 이미지 갯수
!ls -l ./IMAGE/Positive/ | grep jpg | wc -l

In [ ]:
# 이미지 패스 지정
path = './IMAGE/Negative/00001.jpg'

In [ ]:
# 이미지 읽어오기
# Reads the entire file at the given path as a binary string tensor
# Binary data: Information encoded purely as sequences of bits-zeros and ones-that a computer can process directly
# Byte: Fundamental unit of digital storage that consists of exactly eight bits
gfile = tf.io.read_file(path)
image = tf.io.decode_image(gfile)

In [ ]:
# 이미지 shape 확인
image.shape

In [ ]:
# 읽어온 이미지 보기
plt.imshow(image)
plt.show()

### glob과 from_tensor_slices , Pipeline 이용하여 이미지 데이터셋 만들기
+ glob활용하여 이미지 패스를 리스트 형태로 만들기
+ from_tensor_slices 활용하여 이미지 패스 리스트를 Dataset 으로 만들기
+ Pipeline 이용하여 map, cache, Shuffle, batch, prefetch 된 Dataset 만들기

In [ ]:
# glob 활용하여 이미지 패스를 만든다.
# glob 결과로 리스트를 리턴

image_paths = glob('./IMAGE/*/*.jpg')

print(len(image_paths))
print(image_paths[-10:])

In [ ]:
# 이미지 패스를 주면 이미지 읽고 반환하는 함수

def read_image(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    return image

In [ ]:
# 병렬화
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
# 이미지 패스 리스트를 from_tensor_slices 사용하여 데이터셋을 만들고
# map 함수를 사용하여 각 이미지 패스의 이미지들을 병렬로 읽어오기

# tf.data.Dataset.from_tensor_slices: Converts a tensor-like object into a tf.data.Dataset, where each element is a slice (row) of the input
dataset = tf.data.Dataset.from_tensor_slices(image_paths)
dataset = dataset.map(read_image, num_parallel_calls=AUTOTUNE)

In [ ]:
# 데이터셋에서 1개 이미지 가져오기
tf_image = next(iter(dataset))
tf_image.shape

In [ ]:
# Negative 데이터 샘플

plt.imshow(tf_image)
plt.show()

In [ ]:
# 이미지 패스의 이미지 읽고(map) 4개 batch 묶기
dataset = tf.data.Dataset.from_tensor_slices(image_paths)
dataset = dataset.map(read_image)
dataset = dataset.batch(4)

In [ ]:
# 1개의 mini-batch 가져오기
tf_images = next(iter(dataset))
tf_images.shape

In [ ]:
# Negative 데이터 샘플들

for i in range(4):
    plt.imshow(tf_images[i])
    plt.show()

In [ ]:
# from_tensor_slices > map > cache > batch > shuffle > prefetch 형태로 사용

# This converts the Python list image_paths into a tf.data.Dataset, where each element is a single file path string
dataset = tf.data.Dataset.from_tensor_slices(image_paths)  # 입력 : 이미지 패스 리스트
# map applies your custom read_image function to every element, turning each file path into an actual image tensor
dataset = dataset.map(read_image, num_parallel_calls=AUTOTUNE) # 이미지 패스의 각 이미지 읽기
dataset = dataset.cache()
dataset = dataset.batch(4)
dataset = dataset.shuffle(buffer_size=512)
dataset = dataset.prefetch(buffer_size=AUTOTUNE)

In [ ]:
# shuffle 시간 좀 걸림
tf_images = next(iter(dataset))
tf_images.shape

In [ ]:
# shuffle 이전에 첫번째 이미지와 지금은 다른 이미지로 shuffle 되는 것 확인
plt.imshow(tf_images[0])
plt.show()

### Data Preprocess
1. glob 이용하여 이미지 패스 읽기
2. shuffle
3. Train/Test 비율로 나누기
4. 이미지 라벨링 만들기
5. from_tensor_slices > map > cache > batch > shuffle > prefetch 파이프라인 사용하여 이미지/라벨링 데이터셋 만들기

In [ ]:
# Hyperparameter Tunning

num_epochs = 10
batch_size = 32

learning_rate = 0.001
dropout_rate = 0.5

input_shape = (227, 227, 3)  # 사이즈 확인
num_classes = 2              # Postive , Negative

In [ ]:
# glob를 통해 이미지 패스 읽어오기
image_paths_list = glob('./IMAGE/*/*.jpg')

In [ ]:
# 리스트 섞어 주기
image_paths = np.random.permutation(image_paths_list)

In [ ]:
# 샘플 이미지 패스 보기
image_paths[:10]

In [ ]:
# 8: 2 비율로 Train, Test 이미지셋 나누기

TRAIN_SIZE = int(len(image_paths) * 0.8) # 4,800
train_paths = image_paths[:TRAIN_SIZE]
test_paths = image_paths[TRAIN_SIZE:]

In [ ]:
len(train_paths), len(test_paths)

In [ ]:
# Positive, Negative 폴더 이름 반환하는 함수
# EX) ./IMAGE/Negative/07269.jpg --> Negative 가져오는 함수 만들기

def get_class_name(path):
    name = os.path.dirname(path).split('/')[-1]  # 폴더명
    return name

In [ ]:
# get_class_name 함수 정상 동작 여부 확인

for path in train_paths[:4]:
  print(path, get_class_name(path))

In [ ]:
# class 이름 만들기
# np.unique(): Returns the sorted array of distinct values found in the input array
# array: An ordered collection of elements that are stored consecutively in memory
# and accessed by a numeric index starting at zero

train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)
class_names

In [ ]:
# 원핫 인코딩 간단 변환 예제
print( 'Negative' == np.array(['Negative', 'Positive']) )
print( ('Negative' == np.array(['Negative', 'Positive'])).astype(int) )

In [ ]:
# 파일 패스에서 'Negative', 'Positive' 폴더부분을 읽고
# class_name과 비교해서(numpy broadcasting) onehot 만들어 리턴
# tf.cast(x, dtype): Converts a TensorFlow tensor x to the data type specified by dtype

def get_label(path):
    label_name = tf.strings.split(path, '/')[-2]
    onehot = tf.cast(label_name == class_names, tf.uint8)   # One-Hot-Encoding
    # return tf.argmax(onehot)                         # 이번에는 onehot이 아닌 label 번호로
    return onehot                                      # 이번에는 onehot으로

In [ ]:
# 지정된 패스의 이미지를 읽고 rescale하고 , 원핫 인코딩된 class 라벨을 만들어 이미지와 라벨을 리턴

def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale

    label = get_label(path)
    return image, label

In [ ]:
# 이미지 변환 처리 : 여기서는 사용하지 않음

def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [ ]:
# load_image_label 함수 잘 동작하는지 확인
load_image_label('./IMAGE/Negative/00472.jpg')

In [ ]:
# 병렬화
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
# from_tensor_slices > map > cache > batch > shuffle > prefetch 형태로 사용

train_dataset = tf.data.Dataset.from_tensor_slices(train_paths) # 4800
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
#train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(buffer_size=512)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(AUTOTUNE)

In [ ]:
# from_tensor_slices > map > cache > batch > prefetch 형태로 사용
# test set은 shuffle 하지 않음

test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.cache()
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.prefetch(AUTOTUNE)

In [ ]:
# 샘플 이미지 확인

i = 0
for batch_img, batch_label in train_dataset.take(1):
  if i == 0 :
    print(batch_img[i].shape)
    plt.imshow(batch_img[i])
  i = i + 1


In [ ]:
# Functional API 모델 정의

inputs = tf.keras.layers.Input(input_shape)

net = tf.keras.layers.Conv2D(32, (3, 3), padding='SAME')(inputs)  # 227 X 227 X 32
net = tf.keras.layers.Activation('relu')(net)
net = tf.keras.layers.Conv2D(32, (3, 3), padding='SAME')(net)  # 227 X 227 X 32
net = tf.keras.layers.Activation('relu')(net)
net = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(net)  # 113 X 113 X 32
net = tf.keras.layers.Dropout(dropout_rate)(net)

net = tf.keras.layers.Conv2D(64, (3, 3), padding='SAME')(net)  # 113 X 113 X 64
net = tf.keras.layers.Activation('relu')(net)
net = tf.keras.layers.Conv2D(64, (3, 3), padding='SAME')(net)  # 113 X 113 X 64
net = tf.keras.layers.Activation('relu')(net)
net = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(net)  # 56 X 56 X 64
net = tf.keras.layers.Dropout(dropout_rate)(net)

net = tf.keras.layers.Flatten()(net)  # 200,704
net = tf.keras.layers.Dense(512)(net)
net = tf.keras.layers.Activation('relu')(net)
net = tf.keras.layers.Dropout(dropout_rate)(net)
net = tf.keras.layers.Dense(num_classes)(net)
net = tf.keras.layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [ ]:
# 모델 컴파일
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='categorical_crossentropy',  # Loss Function
              metrics=['accuracy'])  # Metrics / Accuracy

In [ ]:
# callback : EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

# ModelCheckpoint
checkpoint_path = "my_checkpoint.ckpt"
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             save_weights_only=True,
                             save_best_only=True,
                             monitor='val_loss',
                             verbose=1)

# ReduceLROnPlateau : val_loss가 2번 이상 감소되지 않으면 lr * factor = lr 새로운 lr로 변경해서 학습 진행
lrReducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.0001)

In [ ]:
# num_epochs = 10
# batch_size = 32
# 데이터 학습시간 오래 걸려, take(10) 사용함 : 정확도 50% 안됨

history = model.fit(
    train_dataset.take(10),
    validation_data=(test_dataset.take(10)),
    epochs=5,
    batch_size=batch_size,
    callbacks=[es, checkpoint, lrReducer]
)

In [ ]:
history.history.keys()

In [ ]:
plt.plot(history.history['accuracy'], label='Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Model Accuracy')
plt.show()

In [ ]:
# 너무 적은 데이터로 학습하니 성능이 50% 정도 나옴

plt.figure(figsize=(16, 30))
for batch_img, batch_label in test_dataset.take(1):
    for i in range(len(batch_img)):
        pred = model.predict(batch_img[i].numpy().reshape(-1,227, 227, 3))
        pred_t = np.argmax(pred)
        plt.subplot(8, 4, i+1)
        plt.title(f'True Value:{np.argmax(batch_label[i])}, Pred Value: {pred_t}')
        plt.imshow(batch_img[i])
        plt.axis('off')